### Indexing
This is developed according to 
https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/online_feature_serving_and_vector_retrieval_bigquery_data_with_feature_store.ipynb

We use FeatureStore as it is the best approach for profduction ready development and online servings.

In [1]:
# Install the packages
!pip3 install --upgrade --quiet google-cloud-aiplatform\
                                 google-cloud-bigquery\
                                 db-dtypes

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.


In [3]:
!pip install protobuf==3.20

In [4]:
#restart kernel
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [23]:
from google.cloud import bigquery
from google.cloud.aiplatform_v1.types import NearestNeighborQuery

In [24]:
from google.cloud import bigquery
from google.cloud.aiplatform_v1.types import NearestNeighborQuery
from vertexai.resources.preview import (FeatureOnlineStore, FeatureView,
                                        FeatureViewBigQuerySource)
from vertexai.resources.preview.feature_store import utils

#set project info
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1" 

bq_client = bigquery.Client(project=PROJECT_ID)


### 1- Create Online Feature Ftore

In [25]:
FEATURE_ONLINE_STORE_ID = "vlt_searchcontent_text_embedding_featurestore"  # @param {type: "string"}
ofs = FeatureOnlineStore.create_optimized_store(FEATURE_ONLINE_STORE_ID)


# get full information of the created feature online store instance
ofs.gca_resource

Creating FeatureOnlineStore
Create FeatureOnlineStore backing LRO: projects/494586852359/locations/us-central1/featureOnlineStores/vlt_searchcontent_text_embedding_featurestore/operations/5778100351773179904
FeatureOnlineStore created. Resource name: projects/494586852359/locations/us-central1/featureOnlineStores/vlt_searchcontent_text_embedding_featurestore
To use this FeatureOnlineStore in another session:
feature_online_store = aiplatform.FeatureOnlineStore('projects/494586852359/locations/us-central1/featureOnlineStores/vlt_searchcontent_text_embedding_featurestore')


name: "projects/494586852359/locations/us-central1/featureOnlineStores/vlt_searchcontent_text_embedding_featurestore"
create_time {
  seconds: 1731477127
  nanos: 651438000
}
update_time {
  seconds: 1731477127
  nanos: 721236000
}
etag: "AMEw9yMJPbHW_vw35_FwY3vuzjM7gWhjHWdsHNE_a1e6tLEEGPRSjCpVTSLh0SD5wgGe"
state: STABLE
dedicated_serving_endpoint {
}
optimized {
}

In [4]:
# Use get to verify the store is created.
FeatureOnlineStore(FEATURE_ONLINE_STORE_ID).gca_resource

name: "projects/494586852359/locations/us-central1/featureOnlineStores/nine_quality_test_text_embedding_featurestore"
create_time {
  seconds: 1731381584
  nanos: 966153000
}
update_time {
  seconds: 1731381585
  nanos: 622762000
}
etag: "AMEw9yOdO-49cFTFrMdxY1eek5xc8Kjc5UpeP1E1Pxh6K3K9lQKYkR05NwAtZBCFzDgw"
state: STABLE
dedicated_serving_endpoint {
}
optimized {
}

### 2-Create Feature View Instance

In [35]:
FEATURE_VIEW_ID = "vlt_textembedding_feature_view1"  # @param {type: "string"}
# A schedule is created based on cron setting.
CRON_SCHEDULE = "TZ=America/Los_Angeles 00 13 11 8 *"  # @param {type: "string"}

In [27]:
# Index configs
DIMENSIONS = 768  # @param {type: "number"}
EMBEDDING_COLUMN = "text_embedding_result"  # @param {type: "string"}
# Optional
LEAF_NODE_EMBEDDING_COUNT = 1000  # @param {type: "number"}
# Optional
#to do: set this later
#CROWDING_COLUMN = "cited_by_filing_date"  # @param {type: "string"}
# Optional
#for multimodal embeddings this can be set to None
FILTER_COLUMNS = ["combined_id","unique_id","headline","description","html_safe_text","startOffset_seconds","endOffset_seconds","fileUri","asset_type"]  # @param


In [32]:
BQ_DATASET_ID='vlt_media_embeddings_integration'
BQ_TABLE_ID='vlt_all_media_content_text_embeddings'
BQ_TABLE_ID_FQN = f"{PROJECT_ID}.{BQ_DATASET_ID}.{BQ_TABLE_ID}"
DATA_SOURCE = f"bq://{BQ_TABLE_ID_FQN}"

big_query_source = FeatureViewBigQuerySource(
    uri=DATA_SOURCE, entity_id_columns=["asset_id"]
)
 

In [46]:
BQ_DATASET_ID='vlt_media_embeddings_integration'
BQ_TABLE_ID='vlt_all_media_content_text_embeddings_vw'
BQ_TABLE_ID_FQN = f"{PROJECT_ID}.{BQ_DATASET_ID}.{BQ_TABLE_ID}"
DATA_SOURCE = f"bq://{BQ_TABLE_ID_FQN}"

big_query_source = FeatureViewBigQuerySource(
    uri=DATA_SOURCE, entity_id_columns=["asset_id"]
)


config = utils.TreeAhConfig(
    leaf_node_embedding_count =LEAF_NODE_EMBEDDING_COUNT
)


index_config = utils.IndexConfig(
    embedding_column=EMBEDDING_COLUMN,
    dimensions=DIMENSIONS,
    crowding_column=None,# to do:  define CROWDING_COLUMN
    #filter_columns=FILTER_COLUMNS, #for multimodal embeddings this can be set to None
    algorithm_config=config,
    distance_measure_type=vertexai.resources.preview.feature_store.utils.DistanceMeasureType.COSINE_DISTANCE
)

print(f"index_config: {index_config}")

nine_fv = ofs.create_feature_view(
    FEATURE_VIEW_ID,
    source=big_query_source,
     # Optional, can be set to None.
    #to do: set to CRON_SCHEDULE
    sync_config=None, #CRON_SCHEDULE
    index_config=index_config,
)

index_config: IndexConfig(embedding_column='text_embedding_result', dimensions=768, algorithm_config=TreeAhConfig(leaf_node_embedding_count=1000), filter_columns=None, crowding_column=None, distance_measure_type=<DistanceMeasureType.COSINE_DISTANCE: 2>)
Creating FeatureView
Create FeatureView backing LRO: projects/494586852359/locations/us-central1/featureOnlineStores/vlt_searchcontent_text_embedding_featurestore/featureViews/vlt_textembedding_feature_view1/operations/5607016342491234304
FeatureView created. Resource name: projects/494586852359/locations/us-central1/featureOnlineStores/vlt_searchcontent_text_embedding_featurestore/featureViews/vlt_textembedding_feature_view1
To use this FeatureView in another session:
feature_view = aiplatform.FeatureView('projects/494586852359/locations/us-central1/featureOnlineStores/vlt_searchcontent_text_embedding_featurestore/featureViews/vlt_textembedding_feature_view1')


In [47]:
#Verify that the FeatureView instance is created by getting the feature view.
FeatureView(
    FEATURE_VIEW_ID, feature_online_store_id=FEATURE_ONLINE_STORE_ID
).gca_resource
     

name: "projects/494586852359/locations/us-central1/featureOnlineStores/vlt_searchcontent_text_embedding_featurestore/featureViews/vlt_textembedding_feature_view1"
create_time {
  seconds: 1731478798
  nanos: 239429000
}
update_time {
  seconds: 1731478798
  nanos: 945522000
}
etag: "AMEw9yMAyuamujXWhFILM5XL-60mPcBEMUiB_Qyw2CMBvAnolCoCRXBldCUbCJkp1FDQ"
big_query_source {
  uri: "bq://nine-quality-test.vlt_media_embeddings_integration.vlt_all_media_content_text_embeddings_vw"
  entity_id_columns: "asset_id"
}
index_config {
  embedding_column: "text_embedding_result"
  embedding_dimension: 768
  distance_measure_type: COSINE_DISTANCE
  tree_ah_config {
    leaf_node_embedding_count: 1000
  }
}

In [48]:
sync_response = nine_fv.sync()


In [ ]:

import time

while True:
    feature_view_sync = nine_fv.get_sync(
        sync_response.resource_name.split("/")[9]
    ).gca_resource
    if feature_view_sync.run_time.end_time.seconds > 0:
        status = "Succeed" if feature_view_sync.final_status.code == 0 else "Failed"
        print(f"Sync {status} for {feature_view_sync.name}. \n {feature_view_sync}")
        # wait a little more for the job to properly shutdown
        time.sleep(30)
        break
    else:
        print("Sync ongoing, waiting for 30 seconds.")
    time.sleep(30)

Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing, waiting for 30 seconds.
Sync ongoing

In [90]:
#for mutimodal embeddings we can ignore this, unless having a description column that want to search through it
# country_filter = NearestNeighborQuery.StringFilter(
#     name="country",
#     allow_tokens=["WIPO (PCT)"],  # try different allow tokens
#     deny_tokens=["United States"],  # try different deny tokens
# )

In [91]:
# EMBEDDINGS = [1] * DIMENSIONS

# result=nine_fv.search(
#     embedding_value=EMBEDDINGS,
#     neighbor_count=10,
#     #string_filters=[country_filter],#for multimodal embedding this can be set to None, unless having a description column
#     return_full_entity=True,  # returning entities with metadata
# )

In [92]:
# result=result.to_dict()

In [93]:
# nearest_neighbours=[]
# for neighbour in result['neighbors']:
#     nearest_neighbour={}
#     nearest_neighbour['entity_id']=neighbour['entity_id']
#     nearest_neighbour['distance']=neighbour['distance']
  
#     for feature in neighbour['entity_key_values']['key_values']['features']:
#         if 'value' in feature:
#             if type(list(feature['value'].values())[0]) is dict:
#                 nearest_neighbour[feature['name']]=[]#list(list(feature['value'].values())[0].values())[0]             
#             else:
#                 nearest_neighbour[feature['name']]=list(feature['value'].values())[0]             
#         else :
#             nearest_neighbour[feature['name']]=None
   
#     nearest_neighbours.append(nearest_neighbour)
    

In [94]:
# nearest_neighbours